# Обработка признаков

В этом домашнем задании вы будете решать задачу предсказания стоимости автомобилей по их различным характеристикам.

In [116]:
import pandas as pd
import numpy as np
RANDOM_STATE = 42

In [174]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/cars_prices.csv", decimal='.')

In [70]:
df.columns

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')

### Описание некоторых признаков

`symboling` - rating corresponds to the degree to which the auto is more risky than its price indicates (+3 more risk and -3 is pretty safe)  
`make` - car types (i.e. car brand)  
`fuel-type` - types of fuel (gas or diesel)  
`aspiration` - engine aspiration (standard or turbo)  
`num-of-doors` - numbers of doors (two or four)  
`body-style` - car body style (sedan or hachback)  
`drive-wheels` - which types of drive wheel (forward-fwd, reversed-rwd)  
`engine-location` - engine mounted location (front or back)  
`wheel-base` - расстояние между осями передних и задних колес  
`length` - car lenght  
`weight` - car weight  
`width` - car width  
`height` - car height  

In [4]:
df.shape

(205, 26)

In [175]:

df.replace('?', np.nan, inplace=True)
df['peak-rpm'] = df['peak-rpm'].astype('float64')
num_cols = df.select_dtypes(include='number').columns
num_cols

Index(['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight',
       'engine-size', 'compression-ratio', 'peak-rpm', 'city-mpg',
       'highway-mpg'],
      dtype='object')

## Заполнение пропусков

Пропуски в этом датасете обозначены как `?`

In [64]:
for c in df.columns:
    print(c, len(df[df[c] == '?']))

AttributeError: 'NoneType' object has no attribute 'columns'

Удалите строки, для которых неизвестно значение price, так как это целевая переменная.

## Вопрос для Quiz

Сколько строк осталось в данных?

In [41]:

# df = df[df['price'] != '?']
# df['price'] #201
df['stroke'].value_counts()[0]

19

Заполните средним значением пропуски в столбцах для числовых признаков и самым популярным значением для категориальных признаков
* `num-of-doors`
* `bore`
* `stroke`
* `horsepower`
* `peak-rpm`

In [176]:

df[num_cols] = df[num_cols].apply(lambda x : x.fillna(x.mean()))
cat_cols = ['num-of-doors', 'bore', 'stroke', 'horsepower']

# df[cat_cols] = df[cat_cols].apply(lambda x : x.fillna(x.value_counts().idxmax()))
# df[cat_cols].isnull().sum()

In [177]:
df[cat_cols] = df[cat_cols].apply(lambda x : x.fillna(x.value_counts().idxmax()))
df[cat_cols].isnull().sum()

num-of-doors    0
bore            0
stroke          0
horsepower      0
dtype: int64

## Вопрос для Quiz

Чему равно среднее значение `peak-rpm` до заполнения пропусков? Ответ округлите до целого числа.

In [180]:
df.shape[1]

26

Пропуски в столбце `normalized-losses` предскажите при помощи линейной регрессии по признакам
`symboling`, `wheel-base`, `length`, `width`, `height`, `curb-weight`, `engine-size`, `compression-ratio`, `city-mpg`, `highway-mpg` и заполните их предсказаниями

In [ ]:
from sklearn.linear_model import LinearRegression
train_df = df[]

## Вопрос для Quiz

Чему равно предсказание линейной регрессии на первом пропущенном значении? Ответ округлите до целого числа.

## 2. Кодирование категориальных признаков

1. Закодируйте бинарные признаки `fuel-type`, `aspiration`, `num-of-doors`, `engine-location` каждый отдельной колонкой, состоящей из 0 и 1.
Единицей кодируйте самую частую категорию.

In [178]:
cols = ['fuel-type', 'aspiration', 'num-of-doors', 'engine-location']
for col in cols:
    df[col] = df[col].apply(lambda x: 1 if x == df[col].value_counts().idxmax() else 0)
df['drive-wheels']

0      rwd
1      rwd
2      rwd
3      fwd
4      4wd
      ... 
200    rwd
201    rwd
202    rwd
203    rwd
204    rwd
Name: drive-wheels, Length: 205, dtype: object

2. Вынесите в переменную `y` целевую переменную `price`, а все остальные колонки - в матрицу `X`.

Закодируйте признаки `make`, `body-style`, `engine-type`, `fuel-system` при помощи LeaveOneOutEncoder.

**Дальше все время работайте с объектами `X`, `y`.**

In [124]:
!pip install category_encoders -q

In [202]:
from category_encoders.leave_one_out import LeaveOneOutEncoder
X = df.drop('price', axis=1)
y = df['price']
coder = LeaveOneOutEncoder()
X[['make', 'body-style', 'engine-type', 'fuel-system']] = coder.fit_transform(X[['make', 'body-style', 'engine-type', 'fuel-system']],y)
X['body-style'].mean()

13207.129353233831

## Вопрос для Quiz

Чему равно среднее значение в столбце `body-style` после кодирования? Ответ округлите до целого числа.

3. Закодируйте признак `drive-wheels` при помощи OHE из библиотеки category_encoders.

In [203]:
from category_encoders.one_hot import OneHotEncoder
coders = OneHotEncoder()
X_new = coders.fit_transform(X[['drive-wheels']], drop='first')
X_new.shape[1]

3

4. В столбце `num-of-cylinders` категории упорядочены по смыслу. Закодируйте их подряд идущими числами, начиная с 1, согласно смыслу.

Подряд идущими числами означает - 1, 2, 3 и так далее без пропусков.

In [200]:
from category_encoders.ordinal import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X['num-of-cylinders'] = ordinal_encoder.fit_transform(X['num-of-cylinders'])
X.shape[1]

28

## Вопрос для Quiz

Сколько столбцов получилось в матрице `X`?

In [201]:
X.shape[1]

28

In [ ]:
X['normalized-losses'] = X['normalized-losses'].astype(float)
X['bore'] = X['bore'].astype(float)
X['stroke'] = X['stroke'].astype(float)
X['horsepower'] = X['horsepower'].astype(float)
X['peak-rpm'] = X['peak-rpm'].astype(float)

y = y.astype(float)

Разбейте данные на тренировочную и тестовую часть в пропорции 3 к 1, зафиксируйте random_state = 42.

In [ ]:
from sklearn.model_selection import train_test_split

# your code here

Масштабируйте данные при помощи MinMaxScaler.

Обучайте масштабирование на тренировочных данных, а потом примените и к трейну, и к тесту.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# your code here

Обучите на тренировочных данных линейную регрессию, сделайте предсказание на тесте и вычислите значение $R^2$ на тестовых данных.

In [ ]:
# your code here

## Вопрос для Quiz

Чему равно значение $R^2$ на тестовых данных? Ответ округлите до сотых.